<a href="https://colab.research.google.com/github/nnugy-21/TensorFlow_Sign_Language_Detection/blob/main/Evaluating_Sign_Detection_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2650, done.
remote: Counting objects: 100% (2650/2650), done.
remote: Compressing objects: 100% (2202/2202), done.
remote: Total 2650 (delta 671), reused 1297 (delta 416), pack-reused 0
Receiving objects: 100% (2650/2650), 32.62 MiB | 28.75 MiB/s, done.
Resolving deltas: 100% (671/671), done.


In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(10, 10),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [ ]:
#Download workspace
#If you have your own workspace you can clone yours instead

import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "TensorFlow_Sign_Language_Detection" in pathlib.Path.cwd().parts:
  while "TensorFlow_Sign_Language_Detection" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('TensorFlow_Sign_Language_Detection').exists():
  !git clone --depth 1 https://github.com/nnugy-21/TensorFlow_Sign_Language_Detection

Cloning into 'TensorFlow_Sign_Language_Detection'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13 (delta 1), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [ ]:
workspace = '/content/TensorFlow_Sign_Language_Detection/workspace'
annotations = workspace + '/annotations'
images = workspace + '/images'
models = workspace + '/models'
pre_trained_models = workspace + '/pre_trained_models'

In [ ]:
#Extracting ASL data in annotations folder
%cd /content/TensorFlow_Sign_Language_Detection/workspace/annotations

!unzip roboflow.zip
!rm roboflow.zip

/content/TensorFlow_Sign_Language_Detection/workspace/annotations
Archive:  roboflow.zip
 extracting: test/Letters.tfrecord   
 extracting: train/Letters.tfrecord  
 extracting: valid/Letters.tfrecord  
 extracting: test/Letters_label_map.pbtxt  
 extracting: train/Letters_label_map.pbtxt  
 extracting: valid/Letters_label_map.pbtxt  
 extracting: README.roboflow.txt     


In [ ]:
# NOTE: Update these TFRecord names and label_map_pbtxt to your file path!
test_record_fname = annotations + '/valid/Letters.tfrecord'
train_record_fname = annotations + '/train/Letters.tfrecord'
label_map_pbtxt_fname = annotations + '/train/Letters_label_map.pbtxt'

In [ ]:
##change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'mobilenet-v2-fpnlite': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 16
    }
}

#in this tutorial we implement the mobilenet-v2-fpnlite
#if you want to scale up to larger efficientdet models you will likely need more compute!
chosen_model = 'mobilenet-v2-fpnlite'

num_steps = 10000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 1000 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [ ]:
#download pretrained models
#in this tutorial we already have the model in /workspace/pre_trained_models folder

%cd /content/TensorFlow_Sign_Language_Detection/workspace/pre_trained_models

import tarfile

tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

/content/TensorFlow_Sign_Language_Detection/workspace/pre_trained_models


In [ ]:
#my_model_path = models + '/' + custom_model_name
my_model_path = '/content/gdrive/MyDrive/Capstone/MY_SSD_MODEL3'
print(my_model_path)

/content/gdrive/MyDrive/Capstone/MY_SSD_MODEL3


In [ ]:
#prepare
#pipeline_fname = my_model_path + '/' + base_pipeline_file
pipeline_fname = '/content/TensorFlow_Sign_Language_Detection/workspace/models/my_ssd_mobnet_v2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config'
#fine_tune_checkpoint = pre_trained_models + '/' + model_name + '/checkpoint/ckpt-0'
fine_tune_checkpoint = '/content/TensorFlow_Sign_Language_Detection/workspace/pre_trained_models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
print(pipeline_fname)
print(fine_tune_checkpoint)

/content/TensorFlow_Sign_Language_Detection/workspace/models/my_ssd_mobnet_v2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
/content/TensorFlow_Sign_Language_Detection/workspace/pre_trained_models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0


In [ ]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd /content/TensorFlow_Sign_Language_Detection/workspace/models/my_ssd_mobnet_v2
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

/content/TensorFlow_Sign_Language_Detection/workspace/models/my_ssd_mobnet_v2
writing custom configuration file


In [ ]:
%cat /content/TensorFlow_Sign_Language_Detection/workspace/models/my_ssd_mobnet_v2/pipeline_file.config

# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 26
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator 

In [ ]:
pipeline_file = '/content/TensorFlow_Sign_Language_Detection/workspace/models/my_ssd_mobnet_v2/pipeline_file.config'

In [ ]:
print(pipeline_file)

/content/TensorFlow_Sign_Language_Detection/workspace/models/my_ssd_mobnet_v2/pipeline_file.config


In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={my_model_path} \
    --checkpoint_dir={my_model_path}

2021-05-21 10:58:14.099525: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
W0521 10:58:16.478784 139644380030848 model_lib_v2.py:1064] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0521 10:58:16.479020 139644380030848 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0521 10:58:16.479089 139644380030848 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0521 10:58:16.479153 139644380030848 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0521 10:58:16.479244 139644380030848 model_lib_v2.py:1085] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2021-05-21 10:58:16.483974: I tensorflow/stream_executor/